<a href="https://colab.research.google.com/github/overfit-ir/persian-twitter-ner/blob/master/ner-survey-models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
!wget 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/persian-ner-twitter-data1.txt'
!wget 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/persian-ner-twitter-data2.txt'
!wget 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/persian-ner-twitter-data3.txt'
!wget 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/persian-ner-twitter-data4.txt'
!wget 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/persian-ner-twitter-data5.txt'
!wget 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/persian-ner-twitter-data6.txt'
!wget 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/persian-ner-twitter-data7.txt'
!wget 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/persian-ner-twitter-data8.txt'
!wget 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/persian-ner-twitter-data9.txt'
!wget 'https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/persian-ner-twitter-data10.txt'

--2021-01-25 06:50:41--  https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/persian-ner-twitter-data1.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 282852 (276K) [text/plain]
Saving to: ‘persian-ner-twitter-data1.txt’

persian-ner-twitter 100%[===================>] 276.22K  --.-KB/s    in 0.04s   

2021-01-25 06:50:42 (7.22 MB/s) - ‘persian-ner-twitter-data1.txt’ saved [282852/282852]

--2021-01-25 06:50:43--  https://raw.githubusercontent.com/overfit-ir/persian-twitter-ner/master/twitter_data/persian-ner-twitter-data/persian-ner-twitter-data2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent

In [3]:
from pathlib import Path
import re

def read_arman(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

In [4]:
texts, tags = read_arman('persian-ner-twitter-data1.txt')

In [5]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 1.8MB 4.4MB/s 
     |████████████████████████████████| 890kB 33.4MB/s 
     |████████████████████████████████| 2.9MB 31.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=6467bffd2bba281893adbdad71d0915af999b144ef5a7b72e9bd9d6b764be548
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 4.1MB/s 


In [6]:
from transformers import AutoConfig, AutoTokenizer, AutoModel, AutoModelForTokenClassification


tokenizer = AutoTokenizer.from_pretrained("m3hrdadfi/albert-fa-base-v2-ner")
model = AutoModelForTokenClassification.from_pretrained("m3hrdadfi/albert-fa-base-v2-ner")
model.eval()


AlbertForTokenClassification(
  (albert): AlbertModel(
    (embeddings): AlbertEmbeddings(
      (word_embeddings): Embedding(80000, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): AlbertTransformer(
      (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
      (albert_layer_groups): ModuleList(
        (0): AlbertLayerGroup(
          (albert_layers): ModuleList(
            (0): AlbertLayer(
              (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
              (attention): AlbertAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bi

In [9]:
from transformers import pipeline
albert_ner = pipeline('ner', model=model, tokenizer=tokenizer)

In [21]:
s = ''
for word in texts[0]:
  s += word + ' '
s

'خرداد 1399 « کیهان » میراث سناتور مصطفی مصباح\u200cزاده 78 ساله شد کیهان لندن تنها رسانه ایرانی در تبعید است که « از جنگ دوم جهانی تاکنون قدمت و یک موسسه غصب شده در تهران » دارد '

In [31]:
tokenizer.tokenize(s)

['▁خرداد',
 '▁',
 '1399',
 '▁',
 '«',
 '▁کیهان',
 '▁',
 '»',
 '▁میراث',
 '▁سناتور',
 '▁مصطفی',
 '▁مصباح',
 '▁زاده',
 '▁',
 '78',
 '▁ساله',
 '▁شد',
 '▁کیهان',
 '▁لندن',
 '▁تنها',
 '▁رسانه',
 '▁ایرانی',
 '▁در',
 '▁تبعید',
 '▁است',
 '▁که',
 '▁',
 '«',
 '▁از',
 '▁جنگ',
 '▁دوم',
 '▁جهانی',
 '▁تاکنون',
 '▁قدمت',
 '▁و',
 '▁یک',
 '▁موسسه',
 '▁غصب',
 '▁شده',
 '▁در',
 '▁تهران',
 '▁',
 '»',
 '▁دارد']

In [32]:
texts[0]

['خرداد',
 '1399',
 '«',
 'کیهان',
 '»',
 'میراث',
 'سناتور',
 'مصطفی',
 'مصباح\u200cزاده',
 '78',
 'ساله',
 'شد',
 'کیهان',
 'لندن',
 'تنها',
 'رسانه',
 'ایرانی',
 'در',
 'تبعید',
 'است',
 'که',
 '«',
 'از',
 'جنگ',
 'دوم',
 'جهانی',
 'تاکنون',
 'قدمت',
 'و',
 'یک',
 'موسسه',
 'غصب',
 'شده',
 'در',
 'تهران',
 '»',
 'دارد']

### Benchmark

In [33]:
!wget "https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py"

--2021-01-25 08:01:08--  https://raw.githubusercontent.com/stefan-it/fine-tuned-berts-seq/master/scripts/preprocess.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 991 [text/plain]
Saving to: ‘preprocess.py’

preprocess.py       100%[===================>]     991  --.-KB/s    in 0s      

2021-01-25 08:01:09 (50.2 MB/s) - ‘preprocess.py’ saved [991/991]



In [37]:
!export MAX_LENGTH=64
!export BERT_MODEL=m3hrdadfi/albert-fa-base-v2-ner

In [38]:
!python3 preprocess.py persian-ner-twitter-data1.txt $BERT_MODEL $MAX_LENGTH > train.txt


Traceback (most recent call last):
  File "preprocess.py", line 6, in <module>
    model_name_or_path = sys.argv[2]
IndexError: list index out of range


In [39]:
!git clone https://github.com/huggingface/transformers
!cd transformers
!pip install transformers/
!pip install -r ./examples/requirements.txt

Cloning into 'transformers'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 60348 (delta 0), reused 2 (delta 0), pack-reused 60340
Receiving objects: 100% (60348/60348), 45.09 MiB | 13.28 MiB/s, done.
Resolving deltas: 100% (42563/42563), done.
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.
ERROR: Could not open requirements file: [Errno 2] No such file or directory: './examples/requirements.txt'


In [46]:
!pip install -r transformers/examples/_tests_requirements.txt

     |████████████████████████████████| 51kB 2.6MB/s 
     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 7.9MB 5.8MB/s 
     |████████████████████████████████| 7.5MB 34.4MB/s 
     |████████████████████████████████| 327kB 59.7MB/s 
     |████████████████████████████████| 163kB 57.8MB/s 
     |████████████████████████████████| 92kB 10.9MB/s 
     |████████████████████████████████| 163kB 45.1MB/s 
     |████████████████████████████████| 112kB 46.9MB/s 
     |████████████████████████████████| 4.5MB 42.4MB/s 
     |████████████████████████████████| 81kB 9.2MB/s 
     |████████████████████████████████| 245kB 56.5MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
     |████████████████████████████████| 122kB 53.5MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=51e94abb11f7febff639a709f7e2c4839c65abe6661e42868247e177f95d5ebb
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e

In [41]:
python3 transformers/examples/token-classification/run_tf_ner.py --data_dir data/ \
--model_type bert \
--labels ./labels.txt \
--model_name_or_path $BERT_MODEL \
--output_dir $OUTPUT_DIR \
--max_seq_length  $MAX_LENGTH \
--num_train_epochs $NUM_EPOCHS \
--per_device_train_batch_size $BATCH_SIZE \
--save_steps $SAVE_STEPS \
--seed $SEED \
--do_train \
--do_eval \
--do_predict

/content


In [48]:
!mv persian-ner* data

In [55]:
!curl -L 'https://drive.google.com/file/d/1Jjhbal535VVz2ap4v4r_rN1UEHTdLK5P/view?usp=sharing' \
| grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > train.txt.tmp
!curl -L 'https://drive.google.com/file/d/1ZfRcQThdtAR5PPRjIDtrVP7BtXSCUBbm/view?usp=sharing' \
| grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > dev.txt.tmp
!curl -L 'https://drive.google.com/file/d/1u9mb7kNJHWQCWyweMDRMuTFoOHOfeBTH/view?usp=sharing' \
| grep -v "^#" | cut -f 2,3 | tr '\t' ' ' > test.txt.tmp

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70329    0 70329    0     0   113k      0 --:--:-- --:--:-- --:--:--  112k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70318    0 70318    0     0   118k      0 --:--:-- --:--:-- --:--:--  118k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 70324    0 70324    0     0   117k      0 --:--:-- --:--:-- --:--:--  117k


In [51]:
!export MAX_LENGTH=128
!export BERT_MODEL=bert-base-multilingual-cased

In [52]:
!python3 preprocess.py train.txt.tmp $BERT_MODEL $MAX_LENGTH > train.txt
!python3 preprocess.py dev.txt.tmp $BERT_MODEL $MAX_LENGTH > dev.txt
!python3 preprocess.py test.txt.tmp $BERT_MODEL $MAX_LENGTH > test.txt

Traceback (most recent call last):
  File "preprocess.py", line 6, in <module>
    model_name_or_path = sys.argv[2]
IndexError: list index out of range
Traceback (most recent call last):
  File "preprocess.py", line 6, in <module>
    model_name_or_path = sys.argv[2]
IndexError: list index out of range
Traceback (most recent call last):
  File "preprocess.py", line 6, in <module>
    model_name_or_path = sys.argv[2]
IndexError: list index out of range


In [56]:
!cat train.txt.tmp

<!DOCTYPE html><html><head><meta name="google" content="notranslate"><meta http-equiv="X-UA-Compatible" content="IE=edge;"><style>@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xIIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:300;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmSU5fBBc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu4mxP.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:700;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmWUlfBBc9.ttf)format('truetype');}</style><meta name="referrer" content="origin"><title>NER-de-train.tsv - Google Drive</title><meta property="og:title" content="NER-de-train.tsv"><meta property="og:type" content="article"><meta property="og:site_name" content="Google Docs"><meta prop